<a href="https://colab.research.google.com/github/sudeepmenon17/PytorchforNN/blob/main/NN_airfoil_S809_combined_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!pip install google-auth

In [17]:
from google.colab import auth
from google.colab import drive

In [18]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
!ls /content/drive/MyDrive/Data_from_Karim/AirfoilS809/

data_set1.csv  data_set2.csv  data_set3.csv  data_set4.csv  neuralNetwork_reg.py  ReadMe.docx


In [20]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [21]:
# Create a Model class
class Model(nn.Module):
  def __init__(self,in_features=5,h1 = 61, h2 = 61, out_features=1):
    super().__init__()
    self.fc1 = nn.Linear(in_features,h1)
    self.fc2 = nn.Linear(h1,h2)
    self.out = nn.Linear(h2,out_features)

  def forward(self,x):
    x = F.sigmoid(self.fc1(x))
    x = F.sigmoid(self.fc2(x))
    x = self.out(x)

    return x

In [22]:
# torch.manual_seed(42)

model = Model()

In [23]:
# Read the data set - data_set3.csv --> Airfoil S809, Re = 2e6, alpha = 14.2 deg
df1 = pd.read_csv("/content/drive/MyDrive/Data_from_Karim/AirfoilS809/data_set1.csv")
df2 = pd.read_csv("/content/drive/MyDrive/Data_from_Karim/AirfoilS809/data_set2.csv")
df3 = pd.read_csv("/content/drive/MyDrive/Data_from_Karim/AirfoilS809/data_set3.csv")
df4 = pd.read_csv("/content/drive/MyDrive/Data_from_Karim/AirfoilS809/data_set4.csv")

In [24]:
# Combine data
frames = [df1, df2, df3, df4]
df = pd.concat(frames)

# Data for beta>1.02 or beta<0.98
df_filtered = df[(df['beta'] > 1.02) | (df['beta'] < 0.98)]

df_filtered

,x,y,beta,feature1,feature2,feature3,feature4,feature5
4223,1.002941,0.000266,1.020826,388.839310,21.900969,0.967255,1.508905,1.967509
4354,1.003335,0.000254,1.022673,416.192842,22.833886,0.966894,1.509316,1.947481
4487,1.003761,0.000240,1.024621,445.422521,23.761253,0.966239,1.504597,1.926424
4622,1.004218,0.000223,1.026653,475.662365,24.719993,0.965617,1.496260,1.906785
4759,1.004711,0.000205,1.028775,506.540421,25.725407,0.965032,1.485008,1.888850
...,...,...,...,...,...,...,...,...
72500,-0.000032,0.000533,0.974411,185.652209,0.006519,0.988927,0.025115,1.629905
72501,-0.000032,0.000664,0.978203,224.328751,0.009889,0.986127,0.032430,1.762057
72854,-0.000040,0.000408,0.977589,186.914237,0.005170,0.987927,0.019519,1.628130
72855,-0.000042,0.000534,0.979770,224.291912,0.007637,0.985172,0.024738,1.758973


In [29]:
from sklearn.preprocessing import StandardScaler

# Preprocess Features
df1 = df_filtered.drop(['x','y'],axis=1)

# # Fill missing values with the mean
# df1 = df1.fillna(df1.mean())


# Standardization
standard_scaler = StandardScaler()
df1_standardized = standard_scaler.fit_transform(df1)


# Features
X = df1_standardized[:,1:]
# Target
y = df1_standardized[:,0]


In [30]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),lr = 0.001)

In [ ]:
#5-Fold Cross Validation
from sklearn import metrics

from sklearn.model_selection import KFold
kFold=KFold(n_splits=3,shuffle=False)

# Initialize a list to store the evaluation scores
scores = []
training_cost = []
testing_cost =[]
iterations = 100


for train_index,test_index in kFold.split(X):
  print("Train Index: ", train_index, "\n")
  print("Test Index: ", test_index)
  print("Size of training data:", len(train_index),"\n")
  print("Size of testing data:", len(test_index),"\n")

  X_train, X_test, y_train, y_test = X[train_index], X[test_index], y[train_index], y[test_index]


  X_train = torch.FloatTensor(X_train)
  X_test = torch.FloatTensor(X_test)

  y_train = torch.FloatTensor(y_train)
  y_test = torch.FloatTensor(y_test)


  losses = []
  for i in range(iterations):
    # go forward and predict
    y_pred = model.forward(X_train)

    # Measure the loss
    loss = criterion(y_pred,y_train)

    # Keep track of the losses
    losses.append(loss.detach().numpy())

    # print every 10 epoch
    if i % 10 == 0:
      print(f'iter: {i} and loss: {loss}')

    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  training_cost.append(losses)

  # Evaluate Model
  with torch.no_grad():
    y_eval = model.forward(X_test)
    test_loss = criterion(y_eval,y_test)
  testing_cost.append(test_loss)
  scores.append(metrics.mean_squared_error(y_eval,y_test))
  print('MSEloss: {}'.format(testing_cost))
  print('MSE error: {}'.format(scores))